## Парсер Яндекс Каталога


In [16]:
import pandas as pd
yaca = pd.read_csv('/home/k.osminin/yaca_base.csv')
yaca.columns

Index([u'Сайт', u'Title', u'ТИЦ', u'Раздел', u'Подр. 2', u'Подр. 3',
       u'Подр. 4', u'Подр. 5', u'Подр. 6', u'Регион', u'Region', u'Reg. 2',
       u'Reg. 3', u'Reg. 4'],
      dtype='object')

In [41]:
yaca.columns = ['Domain','Title','TIC','Section1','Section2','Section3','Section4','Section5','Section6',
               'Region','Region1','Region2','Region3','Region4']
yaca['Fake'] = 1


In [150]:
cols_to_enum = ['Fake','Section1','Section2','Section3','Section4','Section5','Section6']
yaca[cols_to_enum] = cols_to_enum.fillna('na')
ind = [1]
for i in range(len(cols_to_enum)):
    if i > 0:
        ind.append(yaca.groupby(cols_to_enum[:i])[cols_to_enum[i]].unique())


In [285]:
yaca['Section_ind'] = yaca.apply(lambda r: ''.join(['{:02d}'.format(ind[i][tuple(r[cols_to_enum[:i]].values) if i>1 
   else r[cols_to_enum[:1]][0]].tolist().index(r[cols_to_enum[i]])) for i in range(1,len(cols_to_enum))]),axis = 1)


In [305]:
cols_to_enum = ['Fake','Region1','Region2','Region3','Region4']

yaca[cols_to_enum] = cols_to_enum.fillna('na')
ind = [1]
for i in range(len(cols_to_enum)):
    if i > 0:
        ind.append(yaca.groupby(cols_to_enum[:i])[cols_to_enum[i]].unique())
yaca['Region_ind'] = yaca.apply(lambda r: ''.join(['{:02d}'.format(ind[i][tuple(r[cols_to_enum[:i]].values) if i>1 
   else r[cols_to_enum[:1]][0]].tolist().index(r[cols_to_enum[i]])) for i in range(1,len(cols_to_enum))]),axis = 1)


In [417]:
import re
yaca['Domain_proc'] = yaca['Domain'].apply(lambda d: re.findall('https?://(www[0-9s]?\.)?(.+[^/])/?$',d)[0][1])
#d = yaca['Domain'][2]
#re.findall('https?://(www.)*(.+[^/])/?$','https://mondeoclub.ru')[0]

In [418]:

cols_to_export = ['Section_ind','Region_ind','Domain_proc','Title','TIC',u'Section1', u'Section2', u'Section3',
       u'Section4', u'Section5', u'Section6', u'Region', u'Region1',
       u'Region2', u'Region3', u'Region4']
yaca.columns

Index([u'Domain', u'Title', u'TIC', u'Section1', u'Section2', u'Section3',
       u'Section4', u'Section5', u'Section6', u'Region', u'Region1',
       u'Region2', u'Region3', u'Region4', u'Fake', u'Section_ind',
       u'Region_ind', u'Domain_proc'],
      dtype='object')

In [474]:
yaca[cols_to_export].to_csv('yaca_proc.csv', index = False,sep = '\t',header = False)

In [420]:
a = yaca[yaca['Domain_proc'].map(lambda r:'/' in r)][['Domain_proc']]

In [453]:
#a.groupby('domain').count().sort_values(by = 'Domain_proc',ascending = False)

In [475]:
#yaca[yaca['Region_ind'] == '00030000'][['Region1','Region2','Region3','Region4']]
#yaca[yaca['Title'].apply(lambda r: ',' in r)]

#yaca[yaca['Section3']=='12545']

In [ ]:
hive_queries = '''
drop table  user_kposminin.yaca_tmp;
CREATE EXTERNAL TABLE user_kposminin.yaca_tmp(  
  Section_ind string,
  Region_ind string,
  Domain_proc string,
  Title string,
  TIC float,
  Section1 string,
  Section2 string,
  Section3 string,
  Section4 string,
  Section5 string,
  Section6 string,
  Region string,
  Region1 string,
  Region2 string,
  Region3 string,
  Region4 string
)
row format delimited fields terminated by '\t' stored as textfile;
	LOCATION
	  'hdfs://nameservice1/user/kposminin/yaca_tmp'
;
LOAD DATA INPATH '/user/kposminin/yaca_proc.csv' OVERWRITE INTO TABLE user_kposminin.yaca_tmp;

create table user_kposminin.yaca as 
select section_ind, region_ind, domain_proc as site, title, cast(tic as int) as tic
from user_kposminin.yaca_tmp 
;


create table user_kposminin.yaca_region_dict as 
select distinct region_ind, region1, region2, region3, region4, region as region_ru
from user_kposminin.yaca_tmp
where domain_proc <> 'Domain_proc'
order by region_ind
;


create table user_kposminin.yaca_section_dict as 
select distinct section_ind, section1, section2, section3, section4, section5, section6
from user_kposminin.yaca_tmp
where domain_proc <> 'Domain_proc'
order by section_ind
;

create table user_kposminin.yaca_urlfr as
select
  urlfr,
  max(
    named_struct(
      'tic', tic,
      'section_ind',section_ind
    )
  ).section_ind as section_ind,
  max(
    named_struct(
      'tic', tic,
      'region_ind',region_ind
    )
  ).region_ind as region_ind,
  max(
    named_struct(
      'tic', tic,
      'title',title
    )
  ).title as title,
  max(tic) as TIC
from
  (select
    section_ind, 
    region_ind, 
    concat(parse_url(concat('http://', site), 'HOST'), '#', path_fr) as urlfr,
    title,
    tic
  from
    user_kposminin.yaca
    LATERAL VIEW explode(split(parse_url(concat('http://', site), "PATH"), '/')) tt AS path_fr
  ) a
group by urlfr
;

'''

In [483]:
#yaca[yaca['Domain_proc'].apply(lambda r: len(r.split('/')))>2]